In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, SubsetRandomSampler


In [2]:
fireModel = models.resnet50(models.ResNet50_Weights.IMAGENET1K_V1)

c:\Users\maxgl\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
# replace the last layer of the model (called fc in ResNet) -> 3 output classes ["Fire" "NoFire" "Start Fire"]
fireModel.fc = nn.Linear(fireModel.fc.in_features, 3) 

In [4]:
device       = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # define the device to use (cuda:0 if the gpu is availble)
datasetPath  = "big"                                                          # define the path to the dataset (where a folder is a class)
batchSize	 = 32                                                             # define the batch size
trainSplit   = 0.8                                                            # define the ratio of data to use for training
testSplit    = 0.1                                                            # define the ratio of data to use for testing
validSplit   = 0.1                                                            # the rest is used for validation
epochs		 = 100                                                            # define the number of epoch
criterion    = nn.CrossEntropyLoss()                                          # define the loss function
learnRate	 = 0.01                                                           # define the learning rate
optimizer    = optim.Adam(fireModel.parameters(), lr=learnRate)                # define the optimizer
imgSize      = 224                                                            # define the input size

In [5]:
# Preparing data transformations (composition of transforms)
transform   = transforms.Compose([
								transforms.RandomResizedCrop(imgSize),	# Resize the picture
								transforms.ToTensor()					# Transform the picture as a tensor
])
# Preparing target (label) transformations (composition of transforms)
target_transform = transforms.Compose([
		# Only a function that 1) transform an int (label) to a tensor, 2) encode the tensor (one-hot), 3) convert the tensor to float
		lambda label: torch.nn.functional.one_hot(torch.tensor( label ), num_classes=3).float()
])
# Create the dataset based on the path and apply the transformations defined above
dataset   = datasets.ImageFolder(
									datasetPath,
									transform=transform,
									target_transform=target_transform
)

# Compute the number of pictures that will be used for train, val and test datasets
trainlen	= int(trainSplit * len(dataset))
testlen		= int(testSplit  * len(dataset))
validlen	= len(dataset) - trainlen - testlen
# Then random split indices of the dataset into subsets: train, valid and test
# The results are sampler (dataset indices generator)
trainset,validset,testset = torch.utils.data.random_split(range(len(dataset)), [trainlen, validlen, testlen])

# create a loader foreach subset based on its sampler with a batch_size
trainloader = DataLoader(
						dataset,
						batch_size=batchSize,
						sampler=torch.utils.data.SubsetRandomSampler(trainset), # Use the SubsetRandomSampler to random indices at each epoch
)

valloader = DataLoader(
						dataset,
						batch_size=batchSize,
						sampler=validset
)

testloader = DataLoader(
						dataset,
						batch_size=batchSize,
						sampler=testset
)


In [6]:
fireModel = fireModel.to(device) # Move the model to the GPU or CPU if no GPU are available

# Train the model for each epoch
for epoch in range(epochs):
	# Training
	fireModel.train(True)			# set the model in training mode
	totalAccuracy = 0.0				# manually compute accuracy, set 0% as inital accuracy
	totalLoss = 0.0						# manually compute the loss, set 0% as inital loss
	counter = 0								# count number of data during an epoch (to divide accuracy and loss)
	for (i, data) in enumerate(trainloader): 									# for each $i$th batch of data in the trainset
		inputs, labels	= data																	# put the picture tensor into $inputs$ and labels into $labels$
		inputs, labels	= inputs.to(device), labels.to(device)	# move inputs and labels to the device (GPU/CPU)
		optimizer.zero_grad()																		# reset the gradient
		outputs					= fireModel(inputs)											# Forward pass (predict) the ouputs
		loss						= criterion(outputs, labels)						# Compute the loss
		loss.backward()																					# Compute updates of parameters
		optimizer.step()																				# Update the parameters
		accuracy				= outputs.argmax(dim=1) == labels.argmax(dim=1)	# Compute the batch accuracy (when outpus match the labels)
		totalAccuracy  += accuracy.int().sum()										# add the batch accuracy to the total accuracy
		totalLoss 	   += loss.item()															# do the same with the loss
		counter				 += len(inputs)															# increase the counter
	# Print loss and accuracy
	print(f'train [{epoch + 1}] loss: {totalLoss / counter:.3f} accuracy: {100 * totalAccuracy / counter:.3f}%')
	# Validate
	fireModel.train(False)	# Disable training mode (useful when dropout)
	with torch.no_grad():		# Disable autograd (speedup)
		totalAccuracy	= 0.0
		totalLoss		= 0.0
		counter					= 0
		for (i, data) in enumerate(valloader):
			inputs, labels	= data
			inputs, labels	= inputs.to(device), labels.to(device)
			outputs					= fireModel(inputs)
			loss						= criterion(outputs, labels)
			accuracy				= outputs.argmax(dim=1) == labels.argmax(dim=1)
			totalAccuracy += accuracy.int().sum()
			totalLoss	 += loss.item()
			counter				 += len(inputs)
		print(f'validation [{epoch + 1}] loss: {totalLoss / counter:.3f} accuracy: {100 * totalAccuracy / counter:.3f}%')
		torch.save(fireModel, f'FireResNet50-{epoch + 1:d}.pt')
torch.save(fireModel, 'FireResNet50.pt')


train [1] loss: 0.028 accuracy: 74.669%
validation [1] loss: 0.017 accuracy: 80.840%
train [2] loss: 0.017 accuracy: 79.907%
validation [2] loss: 0.015 accuracy: 82.521%
train [3] loss: 0.016 accuracy: 80.034%
validation [3] loss: 0.014 accuracy: 84.874%
train [4] loss: 0.016 accuracy: 80.770%
validation [4] loss: 0.017 accuracy: 79.832%
train [5] loss: 0.015 accuracy: 82.306%
validation [5] loss: 0.014 accuracy: 84.034%
train [6] loss: 0.014 accuracy: 82.285%
validation [6] loss: 0.014 accuracy: 84.874%
train [7] loss: 0.014 accuracy: 83.358%
validation [7] loss: 0.017 accuracy: 84.034%
train [8] loss: 0.013 accuracy: 83.926%
validation [8] loss: 0.014 accuracy: 86.723%
train [9] loss: 0.013 accuracy: 84.178%
validation [9] loss: 0.012 accuracy: 86.387%
train [10] loss: 0.012 accuracy: 85.420%
validation [10] loss: 0.014 accuracy: 83.361%
train [11] loss: 0.012 accuracy: 85.104%
validation [11] loss: 0.019 accuracy: 79.664%
train [12] loss: 0.012 accuracy: 85.336%
validation [12] loss

In [8]:
fireModel = torch.load('FireResNet50-97.pt')
fireModel.train(False)
with torch.no_grad():
  totalAccuracy	= 0.0
  totalLoss		= 0.0
  counter					= 0
  for (i, data) in enumerate(testloader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    # forward
    outputs = fireModel(inputs)
    loss = criterion(outputs, labels)
    accuracy	= outputs.argmax(dim=1) == labels.argmax(dim=1)
    totalAccuracy += accuracy.int().sum()
    totalLoss += loss.item()
    counter += len(inputs)
  print(f'Test loss: {totalLoss / counter:.3f} accuracy: {100 * totalAccuracy / counter:.3f}%')

c:\Users\maxgl\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Test loss: 0.009 accuracy: 88.552%
